In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset= pd.read_csv('coughdf.csv')

In [3]:
updt=dataset.drop('expert_labels_3', inplace=True, axis=1)
updt=dataset.drop('expert_labels_2', inplace=True, axis=1)
updt=dataset.drop('expert_labels_1', inplace=True, axis=1)
updt=dataset.drop('datetime', inplace=True, axis=1)
updt=dataset.dropna(subset=['status'])
updt=dataset.dropna(subset=['gender','respiratory_condition','fever_muscle_pain'],how='all')
updt=updt.replace(to_replace =["other"], 
                            value ="female")
updt=updt.replace(to_replace =["symptomatic"], 
                            value ="COVID-19")


In [4]:
from sklearn import preprocessing
levelencoder= preprocessing.LabelEncoder()
updt['fever_muscle_pain']= levelencoder.fit_transform(updt['fever_muscle_pain'])
updt['gender']= levelencoder.fit_transform(updt['gender'])
updt['respiratory_condition']= levelencoder.fit_transform(updt['respiratory_condition'])
updt['status']= levelencoder.fit_transform(updt['status'])


In [5]:
updt

,Unnamed: 0,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status
0,0,0.9609,31.3,34.8,15.0,1,0,0,1
1,1,0.9609,31.3,34.8,15.0,1,0,0,1
2,2,0.9301,40.0,-75.1,34.0,1,1,0,1
4,4,0.9968,NaN,NaN,21.0,1,0,0,1
6,6,0.0306,13.8,-89.6,NaN,0,0,1,0
...,...,...,...,...,...,...,...,...,...
19983,19983,0.9820,19.0,72.8,32.0,1,0,0,1
19985,19985,0.6354,41.0,28.7,35.0,0,0,0,1
19986,19986,0.9866,NaN,NaN,40.0,0,0,0,1
19989,19989,0.5634,41.9,60.2,NaN,0,0,1,0


training CNN


In [6]:
m = updt.iloc[:, 1:].values

In [7]:
m.shape

(11259, 8)

In [8]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(m[:, 1:4])
m[:, 1:4] = imputer.transform(m[:, 1:4])

In [9]:
def split_sequences(sequences, n_steps):
	a, b = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
    
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		a.append(seq_x)
		b.append(seq_y)
	return array(a), array(b)

In [10]:
from numpy import array
a, b = split_sequences(m, 8)

In [11]:
a.shape

(11252, 8, 7)

In [12]:
b

array([0., 1., 0., ..., 1., 0., 1.])

In [13]:
from sklearn.model_selection import train_test_split
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size = 0.2, random_state = 1)

In [14]:
n_features = a.shape[2]

In [15]:
n_features

7

In [16]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM
import tensorflow as tf


In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(a_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay =  1e-5)
model.compile(loss= 'mse' ,optimizer=opt, metrics=['accuracy'])




#model.compile(optimizer='adam', loss='sparse_categorical_cr',metrics = ['accuracy'])
# fit model
model.fit(a_train,  b_train,  epochs=1000, validation_data=(a_test,b_test))

Epoch 1/1000
282/282 [==============================] - 10s 26ms/step - loss: 0.2278 - accuracy: 0.7500 - val_loss: 0.1868 - val_accuracy: 0.7530
Epoch 2/1000
282/282 [==============================] - 7s 24ms/step - loss: 0.1848 - accuracy: 0.7575 - val_loss: 0.1820 - val_accuracy: 0.7526
Epoch 3/1000
282/282 [==============================] - 7s 24ms/step - loss: 0.1806 - accuracy: 0.7570 - val_loss: 0.1845 - val_accuracy: 0.7526
Epoch 4/1000
282/282 [==============================] - 7s 24ms/step - loss: 0.1784 - accuracy: 0.7572 - val_loss: 0.1774 - val_accuracy: 0.7521
Epoch 5/1000
282/282 [==============================] - 7s 25ms/step - loss: 0.1765 - accuracy: 0.7572 - val_loss: 0.1797 - val_accuracy: 0.7526
Epoch 6/1000
282/282 [==============================] - 7s 25ms/step - loss: 0.1728 - accuracy: 0.7575 - val_loss: 0.1746 - val_accuracy: 0.7526
Epoch 7/1000
282/282 [==============================] - 7s 25ms/step - loss: 0.1707 - accuracy: 0.7600 - val_loss: 0.1740 - val_a

In [ ]:
test_loss, test_acc = model.evaluate(a_test,b_test)
print('test_acc: ',test_acc)